In [2]:
import datasets

/workspace-SR006.nfs2/Bulatov_A/env_main/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def extract_answer(text):
    split_pattern = '####'
    if split_pattern not in text:
        return text.strip().replace(',', '')
    else:
        _, ans = text.strip().split('####', 1)
        ans = '####' + ans
        ans = ans.strip().replace(',', '')
        return ans

def extract_cot(text):
    split_pattern = '####'
    if split_pattern not in text:
        return None
    else:
        cot, _ = text.strip().split('####', 1)
        cot = cot.strip()
        return cot

In [4]:
from datasets import Dataset


def get_hf_dataset(file_path):
    with open(file_path, encoding="utf-8") as f:
        lines = [line.split('||') for line in f.read().splitlines() if (len(line) > 0 and not line.isspace()
                                                                            and len(line.split('||')) ==2 )]
    src_lines, tgt_lines = list(zip(*lines))
    src_lines = list(src_lines)
    tgt_lines = list(tgt_lines)

    srcs, tgts, cots = [], [], []
    for src, tgt in zip(src_lines, tgt_lines):
        #import pdb; pdb.set_trace()
        ans = extract_answer(tgt)[5:]
        cot = extract_cot(tgt)
        tgt = tgt.split('#### ')[1]

        srcs.append(src)
        tgts.append(tgt)
        cots.append(cot)
    ds = Dataset.from_dict(dict(task=srcs, labels=tgts, cot=cots))

    return ds


### 4x4

In [5]:
train_path = '/workspace-SR006.nfs2/Bulatov_A/rmt/tools/implicit_chain_of_thought/data/4_by_4_mult/train.txt'
val_path = '/workspace-SR006.nfs2/Bulatov_A/rmt/tools/implicit_chain_of_thought/data/4_by_4_mult/valid.txt'

In [73]:
ds_train = get_hf_dataset(train_path)
ds_val = get_hf_dataset(val_path)

In [ ]:
hf_dataset = datasets.DatasetDict(train=ds_train, valid=ds_val)
hf_dataset.push_to_hub('booydar/multiplication_4x4', token="your_token_here")

Uploading the dataset shards: 100%|██████████| 1/1 [00:00<00:00,  1.31it/s]


CommitInfo(commit_url='https://huggingface.co/datasets/booydar/multiplication_4x4/commit/e224243bb9970a0937976173287d24816d683ff9', commit_message='Upload dataset', commit_description='', oid='e224243bb9970a0937976173287d24816d683ff9', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/booydar/multiplication_4x4', endpoint='https://huggingface.co', repo_type='dataset', repo_id='booydar/multiplication_4x4'), pr_revision=None, pr_num=None)

### 5x5

In [ ]:
train_path = '/workspace-SR006.nfs2/Bulatov_A/rmt/tools/implicit_chain_of_thought/data/5_by_5_mult/train.txt'
val_path = '/workspace-SR006.nfs2/Bulatov_A/rmt/tools/implicit_chain_of_thought/data/5_by_5_mult/valid.txt'

ds_train = get_hf_dataset(train_path)
ds_val = get_hf_dataset(val_path)

hf_dataset = datasets.DatasetDict(train=ds_train, valid=ds_val)
hf_dataset.push_to_hub('booydar/multiplication_5x5', token="your_token_here")

Uploading the dataset shards: 100%|██████████| 1/1 [00:00<00:00,  1.27it/s]


CommitInfo(commit_url='https://huggingface.co/datasets/booydar/multiplication_5x5/commit/0460ea5f5468419ce40dd642de25a3a7b896daaa', commit_message='Upload dataset', commit_description='', oid='0460ea5f5468419ce40dd642de25a3a7b896daaa', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/booydar/multiplication_5x5', endpoint='https://huggingface.co', repo_type='dataset', repo_id='booydar/multiplication_5x5'), pr_revision=None, pr_num=None)

### gsm8k

In [ ]:
train_path = '/workspace-SR006.nfs2/Bulatov_A/rmt/tools/implicit_chain_of_thought/data/gsm8k/train.txt'
val_path = '/workspace-SR006.nfs2/Bulatov_A/rmt/tools/implicit_chain_of_thought/data/gsm8k/valid.txt'
test_path = '/workspace-SR006.nfs2/Bulatov_A/rmt/tools/implicit_chain_of_thought/data/gsm8k/test.txt'
train_no_aug = '/workspace-SR006.nfs2/Bulatov_A/rmt/tools/implicit_chain_of_thought/data/gsm8k/train_no_aug.txt'

ds_train = get_hf_dataset(train_path)
ds_val = get_hf_dataset(val_path)
ds_test = get_hf_dataset(test_path)
ds_train_no_aug = get_hf_dataset(train_no_aug)

hf_dataset = datasets.DatasetDict(train=ds_train, valid=ds_val, test=ds_test, train_no_aug=ds_train_no_aug)
hf_dataset.push_to_hub('booydar/gsm8k', token="your_token_here")

Uploading the dataset shards: 100%|██████████| 1/1 [00:01<00:00,  1.51s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/booydar/gsm8k/commit/71eb04c3261d8da50bae2dccdec5d70dace5d07d', commit_message='Upload dataset', commit_description='', oid='71eb04c3261d8da50bae2dccdec5d70dace5d07d', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/booydar/gsm8k', endpoint='https://huggingface.co', repo_type='dataset', repo_id='booydar/gsm8k'), pr_revision=None, pr_num=None)

In [77]:
hf_dataset['train'][0]

{'task': 'Jen shared a pack of chocolates among her friends. She gave 20% to Lucy, 30% to Sarah and the remaining were shared equally among four others. If the pack contained 100 chocolates, how many chocolates were each of the four others getting?',
 'labels': '12.5',
 'cot': '<<20+30=50>> <<100-50=50>> <<100*50/100=50>> <<50/4=12.5>>'}

### tokenize

In [6]:
hf_dataset = datasets.load_dataset("booydar/multiplication_4x4")

In [9]:
import torch
from torch.nn.utils.rnn import pad_sequence

In [10]:
from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelForCausalLM.from_pretrained("gpt2")

In [11]:
class Holder:
    def __init__(self):
        pass

args = Holder()
args.use_cot = False
args.num_mem_tokens = None

In [82]:
id_pad_value = tokenizer.pad_token_id if tokenizer.pad_token_id is not None else tokenizer.eos_token_id
# id_pad_value = -100


if args.use_cot in (False, None):
    inputs_key = 'examples_nocot'
    labels_key = 'labels_nocot'
else:
    inputs_key = 'examples_all'
    labels_key = 'labels_all'
    
def collate_fn(batch):
    input_ids = [torch.tensor(b[inputs_key]) for b in batch]
    labels = [torch.tensor(b[labels_key]) for b in batch]
    attention_mask = [torch.ones_like(b, dtype=int) for b in input_ids]
    # labels_mask defines which input_ids participate in loss calculation
    labels_mask = [torch.sign(torch.tensor(b[labels_key])) for b in batch]


    input_ids = pad_sequence(input_ids, padding_value=id_pad_value, batch_first=True)
    labels = pad_sequence(labels, padding_value=id_pad_value, batch_first=True)
    attention_mask = pad_sequence(attention_mask, padding_value=0, batch_first=True)
    labels_mask = pad_sequence(labels_mask, padding_value=0, batch_first=True)

    collated = {'input_ids': input_ids,
                'labels': labels, 
                'attention_mask': attention_mask,
                }
    if args.num_mem_tokens is not None:
        # add labels mask only for RMT, ARMT
        collated['labels_mask'] = labels_mask.bool()
    return collated

In [83]:
train_dataset = hf_dataset['train']

In [84]:
batch = [train_dataset[i] for i in range(10)]

In [85]:
batch[0]

{'task': '1 3 3 8 * 5 1 0 5',
 'labels': '5 6 9 9 7 7 1 4',
 'cot': '5 5 6 1 4 + 0 1 3 3 8 0 ( 5 6 9 4 2 1 ) + 0 0 0 0 0 0 0 ( 5 6 9 4 2 1 0 ) + 0 0 0 5 5 6 1 4'}

In [102]:
think = tokenizer.bos_token_id
ans = tokenizer.bos_token_id
eos = tokenizer.eos_token_id

In [142]:
args.num_mem_tokens = 10
args.use_cot = True

In [ ]:
think = tokenizer.bos_token_id
ans = tokenizer.bos_token_id
eos = tokenizer.eos_token_id

def collate_fn(batch):
    input_ids, labels, labels_mask, attention_mask = [], [], [], []
    for sample in batch:
        task, lab, cot = sample['task'], sample['labels'], sample['cot']
        task_tokens = tokenizer.encode(task, add_special_tokens=False)
        labels_tokens = tokenizer.encode(lab, add_special_tokens=False)
        cot_tokens = tokenizer.encode(cot, add_special_tokens=False)

        if args.use_cot:
            full_input = task_tokens + [think] + cot_tokens + [ans] + labels_tokens + [eos]
        else:
            full_input = task_tokens + [ans] + labels_tokens + [eos]
        inp_ids = torch.tensor(full_input)
        input_ids.append(inp_ids)

        lab = torch.tensor(full_input)
        lab[:len(task_tokens)] = -100
        labels.append(lab)

        lab_mask = torch.ones_like(inp_ids)
        lab_mask[:len(task_tokens)] = 0
        labels_mask.append(lab_mask)
        attention_mask.append(torch.ones_like(inp_ids))
        
    input_ids = pad_sequence(input_ids, padding_value=id_pad_value, batch_first=True)
    attention_mask = pad_sequence(attention_mask, padding_value=0, batch_first=True)
    labels = pad_sequence(labels, padding_value=id_pad_value, batch_first=True)
    labels_mask = pad_sequence(labels_mask, padding_value=0, batch_first=True)

    collated = {'input_ids': input_ids,
                'labels': labels, 
                'attention_mask': attention_mask,
                }
    if args.num_mem_tokens is not None:
        # add labels mask only for RMT, ARMT
        collated['labels_mask'] = labels_mask.bool()
    return collated

In [144]:
collated = collate_fn(batch)

In [145]:
tokenizer.decode(collated['input_ids'][0])

'1 3 3 8 * 5 1 0 5<|endoftext|>5 5 6 1 4 + 0 1 3 3 8 0 ( 5 6 9 4 2 1 ) + 0 0 0 0 0 0 0 ( 5 6 9 4 2 1 0 ) + 0 0 0 5 5 6 1 4<|endoftext|>5 6 9 9 7 7 1 4<|endoftext|>'

In [146]:
tokenizer.decode(collated['input_ids'][0][collated['labels_mask'][0]])

'<|endoftext|>5 5 6 1 4 + 0 1 3 3 8 0 ( 5 6 9 4 2 1 ) + 0 0 0 0 0 0 0 ( 5 6 9 4 2 1 0 ) + 0 0 0 5 5 6 1 4<|endoftext|>5 6 9 9 7 7 1 4<|endoftext|>'

In [147]:
tokenizer.decode(collated['labels'][0][collated['labels_mask'][0]])

'<|endoftext|>5 5 6 1 4 + 0 1 3 3 8 0 ( 5 6 9 4 2 1 ) + 0 0 0 0 0 0 0 ( 5 6 9 4 2 1 0 ) + 0 0 0 5 5 6 1 4<|endoftext|>5 6 9 9 7 7 1 4<|endoftext|>'

### Metrics

In [195]:
think_text = tokenizer.decode(think)
ans_text = tokenizer.decode(ans)

def extract_cot(text):
    try:
        start_index = text.index(think_text)
        end_index = text.index(ans_text, start_index + len(think_text))
        return text[start_index + len(think_text):end_index]
    except ValueError as e:
        return ''

def extract_answer(text):
    return text.split(ans_text)[-2]
        
def compute_accuracy(eval_pred):
    preds = eval_pred.predictions.argmax(axis=-1)[:, :-1]
    labels = eval_pred.label_ids[:, 1:]
    print("preds.shape, labels.shape")
    print(preds.shape, labels.shape)

    labels_masks = labels > 0
    preds_full = [p[m] for p, m in zip(preds, labels_masks)]
    labels_full = [l[m] for l, m in zip(labels, labels_masks)]

    print(len(preds_full), len(labels_full))
    # print(preds_full, labels_full)

    preds_full_text = tokenizer.batch_decode(preds_full, add_special_tokens=True)
    labels_full_text = tokenizer.batch_decode(labels_full, add_special_tokens=True)

    preds_cot = [extract_cot(p) for p in preds_full_text]
    preds_ans = [extract_answer(p) for p in preds_full_text]

    labels_cot = [extract_cot(l) for l in labels_full_text]
    labels_ans = [extract_answer(l) for l in labels_full_text]
    
    acc_cot = np.mean([c == l for c, l in zip(preds_cot, labels_cot)])
    acc_ans = np.mean([c == l for c, l in zip(preds_ans, labels_ans)])

    return {'accuracy_cot': acc_cot, 'accuracy_ans': acc_ans}

In [196]:
labels = collated['labels']
preds = labels

In [197]:
import numpy as np

In [198]:
# preds = eval_pred.predictions.argmax(axis=-1)[:, :-1]
# labels = eval_pred.label_ids[:, 1:]
print("preds.shape, labels.shape")
print(preds.shape, labels.shape)

labels_masks = labels > 0
preds_full = [p[m] for p, m in zip(preds, labels_masks)]
labels_full = [l[m] for l, m in zip(labels, labels_masks)]

print(len(preds_full), len(labels_full))
# print(preds_full, labels_full)

preds_full_text = tokenizer.batch_decode(preds_full, add_special_tokens=True)
labels_full_text = tokenizer.batch_decode(labels_full, add_special_tokens=True)

preds_cot = [extract_cot(p) for p in preds_full_text]
preds_ans = [extract_answer(p) for p in preds_full_text]

labels_cot = [extract_cot(l) for l in labels_full_text]
labels_ans = [extract_answer(l) for l in labels_full_text]

acc_cot = np.mean([c == l for c, l in zip(preds_cot, labels_cot)])
acc_ans = np.mean([c == l for c, l in zip(preds_ans, labels_ans)])

preds.shape, labels.shape
torch.Size([10, 66]) torch.Size([10, 66])
10 10


In [199]:
preds_ans

['5 6 9 9 7 7 1 4',
 '0 4 1 8 0 0 7 7',
 '6 7 4 8 1 3 4 5',
 '2 1 7 5 3 3 8 0',
 '2 1 9 4 0 4 0 1',
 '0 1 3 0 3 7 5 1',
 '8 6 7 8 0 7 9 0',
 '0 2 5 4 2 2 5 1',
 '0 0 6 2 1 6 6 0',
 '6 1 3 9 0 3 2 3']

In [194]:
preds_cot

['|endoftext|>5 5 6 1 4 + 0 1 3 3 8 0 ( 5 6 9 4 2 1 ) + 0 0 0 0 0 0 0 ( 5 6 9 4 2 1 0 ) + 0 0 0 5 5 6 1 4',
 '|endoftext|>0 0 0 0 0 + 0 4 4 2 5 3 ( 0 4 4 2 5 3 ) + 0 0 7 7 6 1 6 ( 0 4 1 0 2 5 6 ) + 0 0 0 8 8 4 0 7',
 '|endoftext|>6 9 9 3 1 + 0 8 8 9 1 4 ( 6 7 8 3 3 4 ) + 0 0 6 8 9 8 4 ( 6 7 4 2 3 3 5 ) + 0 0 0 6 8 9 8 4',
 '|endoftext|>2 3 4 3 3 + 0 8 4 1 0 5 ( 2 1 9 4 3 5 ) + 0 0 8 8 2 2 2 ( 2 1 7 3 6 7 2 ) + 0 0 0 2 7 5 5 0',
 '|endoftext|>2 1 9 4 2 + 0 0 0 0 0 0 ( 2 1 9 4 2 0 ) + 0 0 0 6 7 0 2 ( 2 1 9 0 0 1 2 ) + 0 0 0 4 0 3 8 0',
 '|endoftext|>0 6 3 0 4 + 0 5 4 0 5 0 ( 0 1 8 0 9 0 ) + 0 0 5 4 0 5 0 ( 0 1 3 5 9 5 0 ) + 0 0 0 5 3 1 5 1',
 '|endoftext|>8 9 8 1 1 + 0 7 4 8 7 1 ( 8 6 3 0 9 1 ) + 0 0 4 6 8 5 1 ( 8 6 7 6 7 7 1 ) + 0 0 0 2 3 9 7 0',
 '|endoftext|>0 0 0 0 0 + 0 2 7 8 3 1 ( 0 2 7 8 3 1 ) + 0 0 8 3 1 2 1 ( 0 2 5 2 5 3 1 ) + 0 0 0 2 7 8 3 1',
 '|endoftext|>0 4 8 2 1 + 0 6 7 9 7 1 ( 0 0 6 2 9 1 ) + 0 0 0 4 8 2 1 ( 0 0 6 6 7 4 1 ) + 0 0 0 6 3 1 5 0',
 '|endoftext|>6 8 0 0 4 + 0 

### recurrence

In [12]:
think = tokenizer.bos_token_id
ans = tokenizer.bos_token_id
eos = tokenizer.eos_token_id

def collate_fn(batch):
    input_ids, labels, labels_mask, attention_mask = [], [], [], []
    for sample in batch:
        task, lab, cot = sample['task'], sample['labels'], sample['cot']
        task_tokens = tokenizer.encode(task, add_special_tokens=False)
        labels_tokens = tokenizer.encode(lab, add_special_tokens=False)
        cot_tokens = tokenizer.encode(cot, add_special_tokens=False)

        if args.use_cot:
            full_input = task_tokens + [think] + cot_tokens + [ans] + labels_tokens + [eos]
        else:
            full_input = task_tokens + [ans] + labels_tokens + [eos]
        inp_ids = torch.tensor(full_input)
        input_ids.append(inp_ids)

        lab = torch.tensor(full_input)
        lab[:len(task_tokens)] = -100
        labels.append(lab)

        lab_mask = torch.ones_like(inp_ids)
        lab_mask[:len(task_tokens)] = 0
        labels_mask.append(lab_mask)
        attention_mask.append(torch.ones_like(inp_ids))
        
    input_ids = pad_sequence(input_ids, padding_value=id_pad_value, batch_first=True)
    attention_mask = pad_sequence(attention_mask, padding_value=0, batch_first=True)
    labels = pad_sequence(labels, padding_value=id_pad_value, batch_first=True)
    labels_mask = pad_sequence(labels_mask, padding_value=0, batch_first=True)

    collated = {'input_ids': input_ids,
                'labels': labels, 
                'attention_mask': attention_mask,
                }
    if args.num_mem_tokens is not None:
        # add labels mask only for RMT, ARMT
        collated['labels_mask'] = labels_mask.bool()
    return collated